In [1]:
case class Data(key: String, value: String)

case class ChangeData(key: String, newValue: String, deleted: Boolean, time: Long) {
  assert(newValue != null ^ deleted)
}

In [2]:
val changeDataSource = Seq(
  ChangeData("a", "10", deleted = false, time = 0),
  ChangeData("a", null, deleted = true, time = 1),   // a was updated and then deleted
  ChangeData("b", null, deleted = true, time = 2),   // b was just deleted once
  ChangeData("c", null, deleted = true, time = 3),   // c was deleted and then updated twice
  ChangeData("c", "20", deleted = false, time = 4),
  ChangeData("c", "200", deleted = false, time = 5)
).toDF().createOrReplaceTempView("changes")

// read from last checkpoint 

// val changeDataSource = Seq(
//   ChangeData("d", "82", deleted = false, time = 6),
//   ChangeData("e", "19", deleted = false, time = 7),
//   ChangeData("f", "12", deleted = false, time = 8),
//   ChangeData("g", null, deleted = true, time = 9),
//   ChangeData("d", "21", deleted = false, time = 10)
// ).toDF().createOrReplaceTempView("changes")

In [3]:
val changesDF = spark.sql("select * from changes")

In [5]:
val latestChangeForEachKey = changesDF.selectExpr("key", "struct(time, newValue, deleted) as otherCols").groupBy("key").agg(max("otherCols").as("latest")).selectExpr("key", "latest.*")
latestChangeForEachKey.show()

In [6]:
latestChangeForEachKey.createOrReplaceTempView("lastchanges")

In [7]:
latestChangeForEachKey.write.format("delta").mode("overwrite").saveAsTable("lastchanges") 